In [1]:
import feedparser
from pprint import pprint
from bs4 import BeautifulSoup
import pandas as pd

url = "http://news.google.com/news?q=covid&hl=en-US&sort=date&gl=US&num=10&output=rss"

class ParseFeed():
    def __init__(self, url):
        self.feed_url = url
        
    def clean(self, html):
        '''
        Get the text from html and do some cleaning
        '''
        soup = BeautifulSoup(html)
        text = soup.get_text()
        text = text.replace('\xa0', ' ')
        return text

    def parse(self):
        newsDescription = []
        newsUrl = []
        newsDate = []
        
        '''
        Parse the URL, and print all the details of the news 
        '''        
        feeds = feedparser.parse(self.feed_url).entries
        
        for f in feeds:
            newsDescription.append(self.clean(f.get("description", "")))
            newsUrl.append(f.get("link", ""))
            newsDate.append(f.get("published", ""))
            
        df = pd.DataFrame({'Description':newsDescription, 'URL':newsUrl, 'Date Published':newsDate})            
        return df
    
htmlOutput = ""
feed = ParseFeed(url)
df = feed.parse()

In [2]:
df['Date'] = pd.to_datetime(df['Date Published'])
df = df.head(10).sort_values(by = ['Date'], ascending=False)

# Saving data to html output
htmlOutput = """<html><title>COVID-19 Daily News</title><body><style>
		body {
		  background-image: url('background.jpg');
		  background-repeat: no-repeat;
		  background-attachment: fixed;
		  background-size: cover;
		}
	  </style><center><h1>COVID-19 Daily News</h1></center><dl>"""

for index,row in df.iterrows():
    htmlOutput = htmlOutput + "<dt>" + row['Description'] + "</dt><dd>" + row['Date Published'] + "</dd><dd><a href =" + row['URL'] + ">" + row['URL'] + "</a></dd><br>"
htmlOutput = htmlOutput + "</dl></body></html>"
text_file = open("home.html", "w")
text_file.write(htmlOutput)
text_file.close()

df.to_csv('data.csv', index=False)
df

,Description,URL,Date Published,Date
8,Tracking COVID-19 in Alaska: 120 new cases and...,https://www.adn.com/alaska-news/2020/09/29/tra...,"Tue, 29 Sep 2020 23:05:26 GMT",2020-09-29 23:05:26+00:00
1,Investigational COVID-19 vaccine well-tolerate...,https://www.nih.gov/news-events/news-releases/...,"Tue, 29 Sep 2020 20:24:05 GMT",2020-09-29 20:24:05+00:00
5,Governor Cuomo to Meet with Orthodox Jewish Co...,https://www.governor.ny.gov/news/governor-cuom...,"Tue, 29 Sep 2020 20:01:21 GMT",2020-09-29 20:01:21+00:00
2,"On the road with Operation Warp Speed, the U.S...",https://www.sciencemag.org/news/2020/09/road-o...,"Tue, 29 Sep 2020 19:45:00 GMT",2020-09-29 19:45:00+00:00
3,One number could help reveal how infectious a ...,https://www.sciencemag.org/news/2020/09/one-nu...,"Tue, 29 Sep 2020 19:15:00 GMT",2020-09-29 19:15:00+00:00
9,Daily briefing: One million people have died f...,https://www.nature.com/articles/d41586-020-027...,"Tue, 29 Sep 2020 17:00:07 GMT",2020-09-29 17:00:07+00:00
4,How stigmatizing disease—from COVID-19 to HIV—...,https://www.sciencemag.org/news/2020/09/how-st...,"Tue, 29 Sep 2020 16:00:00 GMT",2020-09-29 16:00:00+00:00
0,COVID-19 Daily Update 9-29-2020 West Virginia...,https://dhhr.wv.gov/News/2020/Pages/COVID-19-D...,"Tue, 29 Sep 2020 14:28:55 GMT",2020-09-29 14:28:55+00:00
7,"As Young Adults Get Infected With Coronavirus,...",https://www.npr.org/sections/health-shots/2020...,"Tue, 29 Sep 2020 12:00:07 GMT",2020-09-29 12:00:07+00:00
6,Global partnership to make available 120 milli...,https://www.who.int/news-room/detail/28-09-202...,"Mon, 28 Sep 2020 18:54:16 GMT",2020-09-28 18:54:16+00:00
